In [1]:
import numpy as np
import matplotlib.pyplot as plt
from ipywidgets import *
import ipywidgets as widgets

In [2]:
def inter_extrapolation(x, y, e):
    """ Extrapolation and interpolation.
    
    :param x: a numpy array
    :param y: a numpy array
    :param e: a numpy array, equivalent of x
    :return: a numpy array
    """
    new_x = np.sort(x)
    new_y = y[np.argsort(x)]

    def point_wise(ep):
        if ep < new_x[0]:
            return new_y[0] + (ep - new_x[0]) * (new_y[1] - new_y[0]) / (new_x[1] - new_x[0])
        elif ep > new_x[-1]:
            return new_y[-1] + (ep - new_x[-1]) * (new_y[-1] - new_y[-2]) / (new_x[-1] - new_x[-2])
        else:
            return np.interp([ep], x, y)[0]
    return np.array([point_wise(i) for i in e])

In [3]:
def calculate_inventory_trading_speed(alpha, phi, t, tt, T, b, k):
    """ For given points t, this function solve for the optimal speed of trading as nu, and investor's inventory along the
        optimal path as Q. 
        This function also returns optimal speed of trading as nut, and investor's inventory along the optimal path Qt as a
        function of time, tt, which is a vector of "infinitely" small time interval.
    """
    tau = T - t
    zeta = ((alpha - 0.5 * b) + np.sqrt(k * phi)) / ((alpha - 0.5 * b) - np.sqrt(k * phi))
    gamma = np.sqrt(phi / k)
    chi = np.sqrt(k * phi) * np.divide(1 + zeta * np.exp(2 * gamma * tau), 1 - zeta * np.exp(2 * gamma * tau))
    Q = np.divide(zeta * np.exp(gamma * tau) - np.exp(-gamma * tau), zeta * np.exp(gamma * T) - np.exp(-gamma * T))
    nu = np.multiply(-chi, Q) / k
    Qt = inter_extrapolation(t, Q, tt)
    nut = inter_extrapolation(t, nu, tt)
    return Q, nu, Qt, nut


def plot_inventory_trading_speed(alpha0, phi, symb, t, tt, T, b, k, labels, main):
    """ This function plots Fig 6.2 using above function to calculate inventory and speed of tading vs time.
    """
    fig, (ax_inv, ax_trad) = plt.subplots(ncols=2)
    fig.set_size_inches(10.5, 5.5)
    color_idx = np.linspace(0, 1, phi.shape[0])
    for i, line in zip(color_idx, range(0, phi.shape[0])):
        inv_line, trad_line, inv_dot, trad_dot = calculate_inventory_trading_speed(alpha0, phi[line], t, tt, T, b, k)
        plt1, = ax_inv.plot(tt, inv_dot, color=plt.cm.rainbow(i), label=labels[line], marker=symb[line], linestyle='None')
        plt2, = ax_trad.plot(tt, trad_dot, color=plt.cm.rainbow(i), label=labels[line], marker=symb[line], linestyle='None')
        plt3, = ax_inv.plot(t, inv_line, linestyle='-', color=plt.cm.rainbow(i))
        plt4, = ax_trad.plot(t, trad_line, linestyle='-', color=plt.cm.rainbow(i))
    ax_inv.legend()
    ax_inv.set_xlabel(r"Time", fontsize=18)
    ax_inv.set_ylabel(r"Inventory", fontsize=18)
    ax_trad.legend()
    ax_trad.set_xlabel(r"Time", fontsize=18)
    ax_trad.set_ylabel(r"Trading Speed", fontsize=18)
    ax_trad.yaxis.set_label_coords(-0.1,0.5)
    plt.suptitle(main, fontsize=20)
    fig.canvas.draw()

In [4]:
def slider_plot_inventory_trading_speed(phi, symb, t, tt, T, b, k, labels):
    """ This function returns a dynamic version of above plot with penalty term alpha controlled by a dynamic slider.
        For demonstration purposes, only alpha = 0.01, and alpha = 100 (as an approximation of positive infinity) are plotted.
        To use this function, call this function on the main file using same argument as above function, but remove 'alpha0' 
        parameter since you do not need to specify alpha here. Then, mannually recompile Liquidation_Permanent_Price_Impact file
        to use the slider.
    """
    def update(alpha):
        fig, (ax_inv, ax_trad) = plt.subplots(ncols=2)
        color_idx = np.linspace(0, 1, phi.shape[0])
        for i, line in zip(color_idx, range(0, phi.shape[0])):
            inv_line, trad_line, inv_dot, trad_dot = calculate_inventory_trading_speed(alpha, phi[line], t, tt, T, b, k)
            plt1, = ax_inv.plot(tt, inv_dot, color=plt.cm.rainbow(i), label=labels[line], marker=symb[line], linestyle='None')
            plt2, = ax_trad.plot(tt, trad_dot, color=plt.cm.rainbow(i), label=labels[line], marker=symb[line], linestyle='None')
            plt3, = ax_inv.plot(t, inv_line, linestyle='-', color=plt.cm.rainbow(i))
            plt4, = ax_trad.plot(t, trad_line, linestyle='-', color=plt.cm.rainbow(i))
        ax_inv.legend()
        ax_inv.set_xlabel(r"Time", fontsize=18)
        ax_inv.set_ylabel(r"Inventory", fontsize=18)
        ax_trad.legend()
        ax_trad.set_xlabel(r"Time", fontsize=18)
        ax_trad.set_ylabel(r"Trading Speed", fontsize=18)
        ax_trad.yaxis.set_label_coords(-0.1,0.5)
        plt.suptitle(r"\alpha = " + str(alpha), fontsize=20)
        fig.canvas.draw()
    return interactive(update, {'manual': False}, alpha=widgets.FloatSlider(min=0.01, max=100, step=0.01, value=alpha0))

In [5]:
def solve_pde(T, dt, qmax, dq, k, a, b, alpha, phi, Ndt, Ndq):
    """ This function solves for optimal trading speed in feed back form as nus, and inventory of optimal path as Qs.
    They are used to be presented as a function of time, t.
    """
    t = np.arange(0, T+dt, dt)
    q = np.arange(0, qmax+dq, dq)

    myleg = []

    nus = np.full((Ndt+1, a.shape[0]), np.NaN)
    Qs = np.full((Ndt+1, a.shape[0]), np.NaN)
    Qs[0,:] = qmax

    for i in range(0, a.shape[0], 1):
        xi = (a[i] * k) / ((1 + a[i]) * k) ** (1 + 1/a[i])

        g = np.full((Ndq+1, Ndt+1), np.NaN)
        nu = np.full((Ndq+1, Ndt+1), np.NaN)
        
        g[:, g.shape[1]-1] = -(alpha ** a[i]) / k ** (a[i] - 1) * np.power(q, 1 + a[i]) + 0.5 * b * np.power(q, 2)

        for j in range(Ndt-1, -1, -1):

            # Explicit Scheme
            dqg = (g[1:g.shape[0], j+1] - g[0:(g.shape[0]-1), j+1]) / dq
            g[1:g.shape[0], j] = g[1:g.shape[0], j+1] + dt * (-phi * np.power(q[1:q.shape[0]], 2) + xi * (np.power(-dqg, 1 + 1 / a[i])))
            g[0, j] = 0

            dqg = (g[1:g.shape[0], j] - g[0:(g.shape[0]-1), j]) / dq
            nu[1:nu.shape[0], j] = np.power(-dqg / ((1 + a[i]) * k), 1 / a[i])
            nu[0, j] = 0

            # Implicit-explicit scheme
            g[:, j] = np.real(g[:, j])
            nu[:, j] = np.real(nu[:, j])

        for j in range(0, Ndt, 1):
            nus[j, i] = inter_extrapolation(q, nu[:, j], [Qs[j, i]])
            Qs[j+1, i] = Qs[j, i] - nus[j, i] * dt

        myleg.append("$a=" + str(a[i]) + "$")
    return nus, Qs, myleg, t, q

In [6]:
def plot_multiple(x, y, xlab=None, ylab=None, main=None, labels=None):
    """ This is a plotting function intended to plot multiple Ys for a single x.
    """
    color_idx = np.linspace(0, 1, y.shape[0])
    for i, line in zip(color_idx, range(0, y.shape[0])):
        if labels is not None:
            plt.plot(x, y[line, :], color=plt.cm.rainbow(i), label=labels[line], linestyle='-')
        else:
            plt.plot(x, y[line, :], color=plt.cm.rainbow(i), linestyle='-')
    if xlab is not None:
        plt.xlabel(xlab, fontsize=18)
    if ylab is not None:
        plt.ylabel(ylab, fontsize=18)
    if main is not None:
        plt.title(main, fontsize=12)
    if labels is not None:
        plt.legend()
    plt.show()